In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### CNVAR

In [128]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import chromATAC as ca
from chromATAC.integrated import IntData
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import matplotlib.gridspec as gridspec
import h5py


RES = [1e06]

def save_collision(omic, layers, coll_key, resolution=RES[0], parent="/Users/mossishahi/Code/lupien/IntData/analysis/experiment100/"):
    path = os.path.join(parent, coll_key)
    d = omic.layers[resolution][coll_key][tuple(sorted(set(layers)))][coll_key]
    if not os.path.exists(path):
        os.makedirs(path)
    for c, g in d.items():
        chromosome_name = ca.info.CHROMOSOMES['names'][c-1]
        n_hits = g['n_hits']
        n_hits.to_csv(f"{path}/{chromosome_name}__n_hits.csv")
        spcf_n_hits = g['spcf_n_hits']
        spcf_n_hits.to_csv(f"{path}/{chromosome_name}__spcf_n_hits.csv")
        mat1_spcf = g['mat1_spcf']
        mat1_spcf.to_csv(f"{path}/{chromosome_name}__spcf_CoTE.csv")
        mat2_spcf = g['mat2_spcf']
        mat2_spcf.to_csv(f"{path}/{chromosome_name}__spcf_CNV.csv")
        exp1_spcf = g['exp1_spcf']
        exp1_spcf.to_csv(f"{path}/{chromosome_name}__E_CoTE_spcf_per_hit.csv")
        exp2_spcf = g['exp2_spcf']
        exp2_spcf.to_csv(f"{path}/{chromosome_name}__E_CNV_spcf_per_hit.csv")
        pval_n_hits = g['pval_n_hits']
        pval_n_hits.to_csv(f"{path}/{chromosome_name}__pval_n_hit.csv")
        qval_n_hits = g['qval_n_hits']
        qval_n_hits.to_csv(f"{path}/{chromosome_name}__qval_n_hit.csv")
        pval_e_spcf = g['pval_e_spcf']
        pval_e_spcf.to_csv(f"{path}/{chromosome_name}__pval_e_spcf.csv")
        qval_e_spcf = g['qval_e_spcf']
        qval_e_spcf.to_csv(f"{path}/{chromosome_name}__qval_e_spcf.csv")
        with h5py.File(f"{path}/{chromosome_name}__hit_spcf.h5", 'w') as f:
            f.create_dataset('array', data=g['hit_spcf'])
        with h5py.File(f"{path}/{chromosome_name}__hit.h5", 'w') as f:
            f.create_dataset('array', data=g['hit_'])

def Clustered_TEs(chromosome_layer, **kwargs):
    resolution = kwargs.get('resolution')
    conditions = {}
    annotations = {}
    resolution = kwargs.get('resolution')
    ann = {0:'TE with no defined CORTE in any chromosome', 1:'TE with defined CORTEs'}
    for c in range(1, 25):
        idx = np.array([i.split('>')[-1] for i in ind.chr.layers[resolution]['TEs']['index'][c]])
        conditions[c] = np.array([int(i in idx) for i in chromosome_layer['index'][c]])
        annotations[c] = ann
    return conditions, annotations

def get_cancertype(chromosome_layer, **kwargs):
    conditions = {}
    annotations = {}
    resolution=kwargs.get('resolution')
    ann = {i:v for i, v in enumerate(tcga_met[' Project'].apply(lambda x:x.split('TCGA-')[-1]).unique())}
    mapping = {v:i for i, v in enumerate(tcga_met[' Project'].apply(lambda x:x.split('TCGA-')[-1]).unique())}
    for c in ca.info.CHROMOSOMES['numericals'].values():
        samples = chromosome_layer['index'][c]
        conditions[c] = [mapping[i.split('_')[0].split('>')[-1]] for i in ind.chr.layers[resolution]['TCGA']['index'][c]]
        annotations[c] = ann
    return conditions, annotations
    
def CTeCore_filter(chromosome_layer, **kwargs):
    conditions = {}
    annotations = {}    
    ann = {0:'TEs with No Significant Difference between #Cores and #Elements in this Chromosome', 1:'TEs with Significant Difference between #Cores and #Elements in this Chromosome'}
    index = kwargs.get('index')
    test_res = kwargs.get('test_res')
    maj = kwargs.get('majority', 3)
    for c in range(1, 25):
        idx = np.array([i.split('>')[-1] for i in index[test_res[:, c-1]>=maj]])
        conditions[c] = np.array([int(i.split('>')[-1] in idx) for i in chromosome_layer['index'][c]])
        annotations[c] = ann
    return conditions, annotations

def GTeCore_filter(genome_layer, **kwargs):
    conditions = {}
    annotations = {}
    ann = {0:'TEs with No Significant Difference between #Cores and #Elements in this Chromosome', 1:'TEs with Significant Difference between #Cores and #Elements in this Chromosome'}
    index = kwargs.get('index')
    test_res = kwargs.get('test_res')
    maj = kwargs.get('majority', 3)
    for c in range(1, 25):
        idx = np.array([i.split('>')[-1] for i in index[test_res[:, c-1]>=maj]])
        conditions[c] = np.array([int(i.split('>')[-1] in idx) for i in genome_layer['index']])
        annotations[c] = ann
    return conditions, annotations

def save_correlation(omic, layers, resolution, c='all', path=".", cmp_method='sim'):
    metrs = ['score', 'pvalue']
    if c=='all':
        c=ca.info.CHROMOSOMES['names']
    if not isinstance(c, list):
        c = [c]
    for c in c:
        path = os.path.join(path, cmp_method+f'/{c}')
        if not os.path.exists(path):
            os.makedirs(path)
        for m in metrs:
            for k, v in omic.layers[resolution]['co'][tuple(sorted(set(layers)))][cmp_method][c][m].items():
                v.to_csv(f'{path}/{c}_{k}_{m}.csv')
def tcga_mapping(name):
    return f"{tcga_met.iloc[np.where([i in name for i in tcga_met['File Name'].apply(lambda x: x.split('_')[0].split('-')[-1])])[0]][' Project'].apply(lambda x:x.split('TCGA-')[-1]).values[0]}_{name}"
    
def TE_family(chromosome_layer, **kwargs):
    conditions = {}
    annotations = {}    
    ann = {i:te.split('>')[-1] for i, te in enumerate(ind.chr.layers[1e06]['TEs']['index'][1])}
    con = {te.split('>')[-1]:i for i, te in enumerate(ind.chr.layers[1e06]['TEs']['index'][1])}
    for c in range(1, 25):
        conditions[c] = np.array([con[i.split('>')[-1]] for i in chromosome_layer['index'][c]])
        annotations[c] = ann
    return conditions, annotations
    
def Cnv_cancertype(chromosome_layer, **kwargs):
    conditions = {}
    annotations = {}
    ann = {i:cancer for i, cancer in enumerate(cnv_meta['cancer_type'].unique())}
    con =  {cancer:i for i, cancer in enumerate(cnv_meta['cancer_type'].unique())}
    for c in tqdm(range(1, 25)):
        samples = chromosome_layer['index'][c]
        conditions[c] = np.array([con[cnv_meta.iloc[np.where(cnv_meta['name']==sample.split('.bed')[0].split(">")[-1])[0]]['cancer_type'].values.item()] for sample in samples if sample.split(">")[-1] in cnv_meta['name'].values])
        annotations[c] = ann
    return conditions, annotations
    
def cnv_gain(matrix, **kwargs):
    m = matrix.copy()
    m[m<=2]=0
    return m
    
def cnv_loss(matrix, **kwargs):
    m = matrix.copy()
    chr = kwargs.get('chr')
    m[m>=2]=0
    return m
    
def cote_present(matrix, **kwargs):
    index = kwargs.get('index')
    df = n_cotes.reindex([i.split('CoTEs>')[-1] for i in index])
    v = df['n_cotes'].values
    m = matrix/v[:, np.newaxis]
    return m
    
def cnv_class_filter(row, **kwargs):
    index_col=kwargs.get('index_col')
    return bool(sample_sig.loc[row.iloc[index_col], row.loc['CN']])

In [13]:
parent_dir = '/Users/mossishahi/Code/lupien/IntData/'

In [14]:
te_dir = parent_dir+'/data/V2-TEs/non_olap'

In [49]:
cote_dir = parent_dir+'data/V2-TEs/cores1000'

In [126]:
n_cotes = pd.read_csv(cote_dir+'/n_clusters_per_TE.tsv', sep='\t', names=['TE', 'n_cotes'])
n_cotes['TE'] = n_cotes['TE'].apply(lambda x: x.split('_Merged')[0])
n_cotes = n_cotes.set_index('TE')

In [16]:
ind = IntData()
ind.add_layer("TEs", input=te_dir, index_mapper=lambda x: x.split("_Merged.bed")[0], resolutions=[1e6])
ind.add_layer("CoTEs", input=cote_dir, index_mapper=lambda x: x.split("_Merged.bed")[0], resolutions=[1e06])
# ind.chr.add_filter('CoTE', Clustered_TEs, ['TEs'], resolutions=[1e05])

loading files from: /Users/mossishahi/Code/lupien/IntData//data/V2-TEs/non_olap


100%|█████████████████████████████████████████████████████████████████████████████████| 977/977 [00:09<00:00, 104.59it/s]


loading files from: /Users/mossishahi/Code/lupien/IntData//data/V2-TEs/non_olap/cores1000


100%|████████████████████████████████████████████████████████████████████████████████| 457/457 [00:00<00:00, 1165.83it/s]


In [ ]:
# ind.add_layer("CNVAR", input=parent_dir+'/data/CNVAR/segments/beds', feature_type='signal', resolutions=[1e05],)
cnv_meta = pd.read_csv(parent_dir + "/data/CNVAR/summary.ascatv3TCGA.penalty70.hg38.tsv", sep='\t')
cnv_sig = pd.read_csv('cnv_sig_reformed.bed', sep='\t')
cnv_sig = cnv_sig[cnv_sig['sample'].apply(lambda x: x in cnv_meta['name'].values)]
cnv_sig['n_copies'] = cnv_sig['n_Major']+cnv_sig['n_Minor']
cnv_sig = cnv_sig.iloc[:,1:10]

sample_sig = pd.read_excel(parent_dir+'data/CNVAR/sample_signature.xlsx', sheet_name='Pancan sig attributions')
sample_sig = sample_sig.set_index('Sample')

In [82]:
ind.add_layer("CNVAR", input=cnv_sig, 
              feature_type='signal', 
              index_col=3, 
              feature_column=8, 
              resolutions=[1e06],
              filter_function = cnv_class_filter)

ind.chr.add_filter('Cancertype', Cnv_cancertype, ['CNVAR'], resolutions=[1e06])

100%|████████████████████████████████████████████████████████████████████████████████████| 24/24 [01:34<00:00,  3.92s/it]


In [129]:
for c in tqdm(
    list(ca.info.CHROMOSOMES['numericals'].values())
    ):
    ind.chr.collide_layers(['CoTEs', 'CNVAR'], resolution=RES[0], groups={'CNVAR':{'Cancertype':list(cnv_meta.cancer_type.unique())}}, chrom=c, kernels = {'CNVAR':cnv_gain, 'CoTEs':cote_present}, coll_key='gain', classifier=lambda x: 'CNVAR' in x, by={'CNVAR':'group', 'CoTEs':'sample'})
    ind.chr.collide_layers(['CoTEs', 'CNVAR'], resolution=RES[0], groups={'CNVAR':{'Cancertype':list(cnv_meta.cancer_type.unique())}}, chrom=c, kernels = {'CNVAR':cnv_loss, 'CoTEs':cote_present}, coll_key='loss', classifier=lambda x: 'CNVAR' in x, by={'CNVAR':'group', 'CoTEs':'sample'})
    save_collision(ind.chr, ['CoTEs', 'CNVAR'], 'gain', resolution=RES[0])
    save_collision(ind.chr, ['CoTEs', 'CNVAR'], 'loss', resolution=RES[0])

 96%|█████████████████████████████████████████████████████████████████████████████▋   | 23/24 [1:16:53<03:20, 200.57s/it]


ValueError: need at least one array to concatenate